In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO
import os

In [2]:
amz_books = pd.read_csv('books_data_with_index.csv')

In [3]:
amz_books.head()

,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount,index
0,Its Only Art If Its Well Hung!,NaN,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],NaN,0
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN,1
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],http://books.google.com/books/content?id=2tsDA...,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,NaN,2000,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,['Religion'],NaN,2
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],http://books.google.com/books/content?id=aRSIg...,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,iUniverse,2005-02,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,['Fiction'],NaN,3
4,"Nation Dance: Religion, Identity and Cultural ...",NaN,['Edward Long'],NaN,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,2003-03-01,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,NaN,4


In [9]:
# number of images expected (add counter to check all images have been pulled)
amz_books['image'].unique().shape[0]

149388

In [11]:
amz_books_100 = amz_books.head(100)

In [15]:
amz_books_100 = amz_books_100.dropna(how="any", subset=["image"])

In [16]:
amz_books_100.isnull().sum()

Title             0
description      13
authors           4
image             0
previewLink       0
publisher        19
publishedDate     1
infoLink          0
categories        7
ratingsCount     63
index             0
dtype: int64

In [ ]:
"""no_images_saved = 0
no_images_not_saved = 0

for index, row in amz_books_100.iterrows():
    url = row["image"]
    image_url = str(url) + ".jpg"

    # Send a GET request to the image URL
    image_response = requests.get(image_url)

    # Check if the request was successful
    if image_response.status_code == 200:
        
        # Save the image to your local machine
        save_name = str(row["index"]) + ".jpg"
        image = Image.open(BytesIO(image_response.content))
        image.save(save_name)
        no_images_saved += 1
    else:
        no_images_not_saved +=1"""

In [17]:
folder_path = 'images'

no_images_saved = 0
no_images_not_saved = 0

for index, row in amz_books_100.iterrows():
    url = row["image"]
    image_url = str(url) + ".jpg"

    # Send a GET request to the image URL
    image_response = requests.get(image_url)

    # Check if the request was successful
    if image_response.status_code == 200:
        
        # Save the image to the specified folder
        save_name = os.path.join(folder_path, str(row["index"]) + ".jpg")
        image = Image.open(BytesIO(image_response.content))
        image.save(save_name)
        no_images_saved += 1
    else:
        no_images_not_saved += 1

In [20]:
amz_books_100.shape

(81, 11)

In [18]:
no_images_saved

81

In [19]:
no_images_not_saved

0

In [21]:
amz_books.shape

(212404, 11)

In [22]:
amz_books = amz_books.dropna(how="any", subset=["image"])

In [23]:
# n.b. 160,329 non Nan values, but I don't think these are all unique, so might need to pull a subset again 
# or exclude certain ids
amz_books.shape

(160329, 11)

In [24]:
amz_books.isnull().sum()

Title                 1
description       23372
authors            5302
image                 0
previewLink           0
publisher         30146
publishedDate       924
infoLink              0
categories         8445
ratingsCount     113202
index                 0
dtype: int64

In [ ]:
folder_path = 'images'

no_images_saved = 0
no_images_not_saved = 0

for index, row in amz_books.iterrows():
    url = row["image"]
    image_url = str(url) + ".jpg"

    # Send a GET request to the image URL
    image_response = requests.get(image_url)

    # Check if the request was successful
    if image_response.status_code == 200:
        
        # Save the image to the specified folder
        save_name = os.path.join(folder_path, str(row["index"]) + ".jpg")
        image = Image.open(BytesIO(image_response.content))
        image.save(save_name)
        no_images_saved += 1
    else:
        no_images_not_saved += 1

In [ ]:
no_images_saved

In [ ]:
no_images_not_saved